### 生成窗寬和窗位分別為[300,50]的資料集

In [1]:
"""
最終更新時間：2020/09/19

更新內容：
* output_annotation_only參數改成output_method，並對該參數的功能做一些調整(增加程式靈活度)。
* 新增「unzip_gzip」這個參數。
* 將各個病患(不含已經排除的病患)的CT影像切片張數儲存成文字檔 --> 每位病患的儲存格式修改為「病患編號 切片張數」。
* 程式碼細節修正。

參數說明：
* kits_path: 存放kits資料集的路徑(裡面裝有各個病患資料的資料夾，必須是已經解壓縮的狀態)。

* save_path: 處理完的資料儲存的路徑，預設為"DATA"。

* testing_set_patient_index: 有哪些編號的病人要在之後做為測試集(編號從0開始算)，
例如testing_set_patient_index = [121, 122, ... 130] (list(range(121,131))) 代表121到130號病人的資料將做為測試集。

* excluded_patient_index: 排除的病患編號(不要輸出哪些病患編號的資料)，
例如使用者打算排除編號為1、11和111的病患資料，則將該參數設成[1,11,111]。預設是無。

* change_annotation_value: 改變標記中的值，
原來標記中的0代表非器官區域；1代表正常器官區域；2代表腫瘤區域，相當將該參數設成[0,1,2]，
如果使用者只打算輸出標記中的腫瘤標記，則將該參數設成[0,0,2]；
如果使用者打算將腫瘤視為正常器官的一部份，則將該參數設成[0,1,1]。預設是無。

* annotation_file_remark: 標記檔案資料夾的註記，
原來該函式生成裝有標記資料的資料夾名稱為"annotations"，
如果將該參數設成"kidney"，該資料夾的名稱將會變成"annotations_kidney"。預設是無。

* window_width_and_window_level: 調整CT影像顯示的窗寬和窗位。如果窗寬和窗位分別是600和100，則輸入[600,100]。

* data_process_and_output_method: 資料處理和輸出的方法(以下將對各種不同的輸出方法做說明)。預設是1。
    * data_process_and_output_method = 1: 對「影像和標記資料」處理和輸出。
    * data_process_and_output_method = 2: 只對「影像資料」處理和輸出。
    * data_process_and_output_method = 3: 只對「標記資料」處理和輸出。

* easy_to_observe_annotations: 是否將標記區域的值調整成方便用肉眼觀察的值([0,1,2] -> [0,128,255]，訓練之前記得要調整回來)。

* unzip_gzip: 是否對資料集路徑中的.gz檔解壓縮(該參數在進行第一次資料前處理的時候才需要設成True，因為KiTS19剛下載完後是.gz的壓縮檔)。

* img_size: 輸出的圖片大小，預設為512*512。
"""

import os
from os import walk
from os.path import join
import gzip
import shutil
import cv2
import nibabel as nib
import numpy as np

def preprocessing_kits(kits_path, 
                       save_path = 'DATA',
                       testing_set_patient_index = None,
                       excluded_patient_index = None,
                       change_annotation_value = None,
                       annotation_file_remark = None,
                       window_width_and_window_level = None,
                       data_process_and_output_method = 1,
                       easy_to_observe_annotations = False,
                       unzip_gzip = False,
                       img_size = 512):
    

    # gzip檔(.gz)解壓縮
    if unzip_gzip:
        for root, dirs, files in walk(kits_path):
            for f in files:
                if 'gz' in f: 
                    if int(join(root, f).split('\\')[1][-3:]) > -1: 
                        # 有時該迴圈會因為EOFError而中斷(原因不明)，因此設上面這個條件式讓程式可以從中斷的地方繼續執行
                        f_in_path = join(root, f)
                        f_out_path = join(root, f)[:-3]

                        with gzip.open(f_in_path, 'rb') as f_in:
                            with open(f_out_path , 'wb') as f_out:
                                shutil.copyfileobj(f_in, f_out)

                        print(f'{f_in_path} 已經解壓縮')

    
    # 建立資料儲存基本路徑的資料夾
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        print('-----建立新資料夾：' + save_path + '-----')
  
    # 以每個病患為單位進行資料前處理
    print('############################### starting data preprocessing - KiTS dataset ###############################')
    
    num_slices_list = [] # 紀錄各個病患的切片張數
        
    case_name_list = []

    for f in os.listdir(kits_path):
        if 'case_' in f:
            case_name_list.append(f)    

    for idx, f in enumerate(case_name_list):
        
        # 判斷目前的病患編號是否在排除的名單當中，如果是就不處理
        skip_process = False
        if excluded_patient_index:
            if int(f[-3:]) in excluded_patient_index:
                skip_process = True

        if not skip_process:
        
            # 判斷目前的病患資料屬於訓練集還是測試集
            if testing_set_patient_index:
                if int(f[-3:]) in testing_set_patient_index:
                    is_in_testing_set = True
                else:
                    is_in_testing_set = False
            else:
                is_in_testing_set = None

            # 讀取該病患的CT影像和標記資料，並轉換成適當的資料型態
            if data_process_and_output_method != 3:
                img = nib.load(join(kits_path, f, 'imaging.nii')).get_fdata()
                img = img.astype('float32')               
                data_len = len(img)

                num_slices_list.append(str(int(f[-3:])) + ' ' + str(img.shape[0]))

            if data_process_and_output_method != 2:
                seg = nib.load(join(kits_path, f, 'segmentation.nii')).get_fdata()
                seg = seg.astype('uint8')
                data_len = len(seg)

        #         # 紀錄該病患CT影像的斜率和截距，作為之後調整像素值至HU值用
        #         slope = img.dataobj.slope
        #         inter = img.dataobj.inter     

        #         # 將CT影像中的像素值根據斜率和截距調整成真正的HU值
        #         img = img * slope + inter

                # 改變標記中的值
                if change_annotation_value:
                    seg_mask = seg.copy()
                    seg[seg_mask == 0] = change_annotation_value[0]
                    seg[seg_mask == 1] = change_annotation_value[1]
                    seg[seg_mask == 2] = change_annotation_value[2]

            # 根據窗寬和窗位調整CT影像HU值顯示的範圍
            if window_width_and_window_level:
                window_width, window_level = window_width_and_window_level
                if data_process_and_output_method != 3:
                    hu_upper_liver = window_level + window_width / 2
                    hu_lower_liver = window_level - window_width / 2
                    img = (img - hu_lower_liver) / (hu_upper_liver - hu_lower_liver) # min_max轉換
                    img[img < 0] = 0
                    img[img > 1] = 1
                    img = img * 255
                    img = img.astype('float32')
                save_path_ = os.path.join(save_path, f'WW{window_width}WL{window_level}')
            else:
                save_path_ = os.path.join(save_path, f'no_windowing')

            # 建立存放該病患處理後的資料的資料夾
            if is_in_testing_set == None:
                if data_process_and_output_method != 3:
                    save_path_images = os.path.join(save_path_, 'images')
                if data_process_and_output_method != 2:
                    if annotation_file_remark:
                        save_path_annotations = os.path.join(save_path_, 'annotations_' + annotation_file_remark)
                    else:
                        save_path_annotations = os.path.join(save_path_, 'annotations')

            elif is_in_testing_set == True:
                if data_process_and_output_method != 3:
                    save_path_images = os.path.join(save_path_, 'test', 'images')
                if data_process_and_output_method != 2:
                    if annotation_file_remark:
                        save_path_annotations = os.path.join(save_path_, 'test', 'annotations_' + annotation_file_remark)
                    else:
                        save_path_annotations = os.path.join(save_path_, 'test', 'annotations')

            elif is_in_testing_set == False:
                if data_process_and_output_method != 3:
                    save_path_images = os.path.join(save_path_, 'train', 'images')
                if data_process_and_output_method != 2:
                    if annotation_file_remark:
                        save_path_annotations = os.path.join(save_path_, 'train', 'annotations_' + annotation_file_remark)
                    else:
                        save_path_annotations = os.path.join(save_path_, 'train', 'annotations')

            if data_process_and_output_method != 3:
                if not os.path.exists(save_path_images):
                    os.makedirs(save_path_images)
                    print('-----建立新資料夾：' + save_path_images + '-----')

            if data_process_and_output_method != 2:
                if not os.path.exists(save_path_annotations):
                    os.makedirs(save_path_annotations)
                    print('-----建立新資料夾：' + save_path_annotations + '-----')

            # 將CT影像和標記資料輸出成png檔到指定的資料夾
            for i in range(data_len):

                # 輸出CT影像 
                if data_process_and_output_method != 3:
                    img_ = img[i,:,:].copy()
                    
                    # 解決case_00160影像大小和其他資料不一致的問題(在「不扭曲原始圖片比例」的條件之下從796x512調整至512x512)
                    if int(f[-3:]) == 160:
                        img_temp = np.zeros((796,796), np.uint8)
                        img_temp[142:654,0:796] = img_
                        img_ = cv2.resize(img_temp, (img_size, img_size)) # 將病患的CT影像統一成設定的大小
                        img_ = img_.astype('float32')
                    
                    if img_.shape != (img_size, img_size):
                        img_ = cv2.resize(img_temp, (img_size, img_size)) # 將病患的CT影像統一成設定的大小
                        img_ = img_.astype('float32')

                    cv2.imwrite(
                        os.path.join(
                            save_path_images, 
                            'KiTS_case' + str(int(f[-3:])).zfill(5) + '_' + str(i).zfill(4) + '.png'), 
                        img_)

                # 輸出標記資料
                if data_process_and_output_method != 2:
                    seg_ = seg[i,:,:].copy()

                    # 解決case_00160影像大小和其他資料不一致的問題(在「不扭曲原始圖片比例」的條件之下從796x512調整至512x512)
                    if int(f[-3:]) == 160:
                        seg_temp = np.zeros((796,796), np.uint8)
                        seg_temp[142:654,0:796] = seg_
                        seg_ = cv2.resize(seg_temp, (img_size, img_size)) # 將病患的標記資料統一成設定的大小
                        seg_ = seg_.astype('uint8')

                    if seg_.shape != (img_size, img_size):
                        seg_ = cv2.resize(seg_, (img_size, img_size)) # 將病患的標記資料統一成設定的大小
                        seg_ = seg_.astype('uint8')

                    if easy_to_observe_annotations:
                        seg_[seg_ == 1] = 128  # 用肉眼方便觀察標記區域(之後要再轉回1才能下去訓練)  
                        seg_[seg_ == 2] = 255  # 用肉眼方便觀察標記區域(之後要再轉回2才能下去訓練)  
                    cv2.imwrite(
                        os.path.join(
                            save_path_annotations, 'KiTS_case' + str(int(f[-3:])).zfill(5) + '_' + str(i).zfill(4) + '.png'), 
                        seg_)               

            # 顯示目前進度
            if (idx + 1) % 10 == 0:
                print(f'目前進度：{idx + 1} / {len(case_name_list)}')     

    # 將屬於測試集的病人編號儲存成文字檔方便觀察
    if testing_set_patient_index:
        with open(os.path.join(save_path, 'patient indices of testing set - KiTS.txt'), 'w') as f:
            for item in testing_set_patient_index:
                if excluded_patient_index:
                    if item not in excluded_patient_index:
                        f.write("%s\n" % item)
                else:
                    f.write("%s\n" % item)

    # 將各個病患(不含已經排除的病患)的CT影像切片張數儲存成文字檔方便觀察(每位病患的儲存格式為「病患編號 切片張數」)
    with open(os.path.join(save_path, 'number of slices of each patient - KiTS.txt'), 'w') as f:
        for item in num_slices_list:
            f.write("%s\n" % item)

## 生成訓練集(影像 only)
* 沒有排除任何的病患資料
* CT影像窗口化：窗寬 = 300，窗口 = 50

In [2]:
import time

st = time.time()

print('程式開始執行...')

preprocessing_kits(
    kits_path = 'kits', 
    testing_set_patient_index = [-1], # 設[-1]而非None可以使系統自動生成train的資料夾
    excluded_patient_index = None,
    change_annotation_value = None,
    annotation_file_remark = None,
    window_width_and_window_level = [300,50],
    data_process_and_output_method = 2, # 只對「影像資料」處理和輸出
    unzip_gzip = False,
    easy_to_observe_annotations = False # 該參數建議設成False，測試程式碼的時候才可以設定成True
)

ed = time.time()

sp_time = ed - st
print(f'花費時間：{sp_time: .4f}秒')

程式開始執行...
-----建立新資料夾：DATA-----
############################### starting data preprocessing - KiTS dataset ###############################
-----建立新資料夾：DATA\WW300WL50\train\images-----
目前進度：10 / 210
目前進度：20 / 210
目前進度：30 / 210
目前進度：40 / 210
目前進度：50 / 210
目前進度：60 / 210
目前進度：70 / 210
目前進度：80 / 210
目前進度：90 / 210
目前進度：100 / 210
目前進度：110 / 210
目前進度：120 / 210
目前進度：130 / 210
目前進度：140 / 210
目前進度：150 / 210
目前進度：160 / 210
目前進度：170 / 210
目前進度：180 / 210
目前進度：190 / 210
目前進度：200 / 210
目前進度：210 / 210
花費時間： 1243.5588秒


## 生成測試集(影像 only)
* 沒有排除任何的病患資料
* CT影像窗口化：窗寬 = 300，窗口 = 50

In [3]:
import time

st = time.time()

print('程式開始執行...')

preprocessing_kits(
    kits_path = 'kits_test', 
    testing_set_patient_index = list(range(210,300)),
    excluded_patient_index = None,
    change_annotation_value = None,
    annotation_file_remark = None,
    window_width_and_window_level = [300,50],
    data_process_and_output_method = 2, # 只對「影像資料」處理和輸出
    #unzip_gzip = True,
    unzip_gzip = False,
    easy_to_observe_annotations = False # 該參數建議設成False，測試程式碼的時候才可以設定成True
)

ed = time.time()

sp_time = ed - st
print(f'花費時間：{sp_time: .4f}秒')

程式開始執行...
############################### starting data preprocessing - KiTS dataset ###############################
-----建立新資料夾：DATA\WW300WL50\test\images-----
目前進度：10 / 90
目前進度：20 / 90
目前進度：30 / 90
目前進度：40 / 90
目前進度：50 / 90
目前進度：60 / 90
目前進度：70 / 90
目前進度：80 / 90
目前進度：90 / 90
花費時間： 492.6417秒


### 從窗寬和窗位為[300,50]的資料集中切出第一階段模型預測出來的腎臟區域

In [23]:
dataset_dir = 'DATA_0919\\WW600WL100'
dataset_dir_new = 'DATA_0930\\WW300WL50'
prediction_1st_dir = 'DATA_0919\\WW600WL100\\train\\annotations_tumor as kidney_predicted'

In [18]:
import os
import glob
import numpy as np

inps = glob.glob(os.path.join(prediction_1st_dir, "*.png"))

preds = []

for i in inps:
    preds.append(cv2.imread(i, cv2.IMREAD_GRAYSCALE))  

preds = np.asarray(preds)

print(preds.shape)

In [24]:
from evaluate import *

st = time.time()

_ = evaluate_model(
    image_dir = os.path.join(dataset_dir_new, 'train', 'images'), 
    label_dir = os.path.join(dataset_dir, 'train', 'annotations_tumor as kidney'), 
    checkpoints_path = None,
    calculate_predicting_indicators = False,
    output_predicted_result = False, 
    segment_out_predicted_region_from_original_images = True, 
    roi_description = 'tumor as kidney', 
    preds = preds,
    batch_process_slice_point = [15041, 29708]) # 根據70位和前140位病患的影像總數作為切點，分批處理資料(避免記憶體不足的問題)

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

Using TensorFlow backend.


----------預測結果資料型態轉換(第1批資料)----------
-----建立新資料夾：DATA_0930\WW300WL50\train\images_only_containing_predicted_roi_tumor as kidney-----
----------開始生成並輸出只包含模型預測區域的圖片(第1批資料)----------
目前進度：第500張照片
目前進度：第1000張照片
目前進度：第1500張照片
目前進度：第2000張照片
目前進度：第2500張照片
目前進度：第3000張照片
目前進度：第3500張照片
目前進度：第4000張照片
目前進度：第4500張照片
目前進度：第5000張照片
目前進度：第5500張照片
目前進度：第6000張照片
目前進度：第6500張照片
目前進度：第7000張照片
目前進度：第7500張照片
目前進度：第8000張照片
目前進度：第8500張照片
目前進度：第9000張照片
目前進度：第9500張照片
目前進度：第10000張照片
目前進度：第10500張照片
目前進度：第11000張照片
目前進度：第11500張照片
目前進度：第12000張照片
目前進度：第12500張照片
目前進度：第13000張照片
目前進度：第13500張照片
目前進度：第14000張照片
目前進度：第14500張照片
目前進度：第15000張照片
----------預測結果資料型態轉換(第2批資料)----------
----------開始生成並輸出只包含模型預測區域的圖片(第2批資料)----------
目前進度：第15500張照片
目前進度：第16000張照片
目前進度：第16500張照片
目前進度：第17000張照片
目前進度：第17500張照片
目前進度：第18000張照片
目前進度：第18500張照片
目前進度：第19000張照片
目前進度：第19500張照片
目前進度：第20000張照片
目前進度：第20500張照片
目前進度：第21000張照片
目前進度：第21500張照片
目前進度：第22000張照片
目前進度：第22500張照片
目前進度：第23000張照片
目前進度：第23500張照片
目前進度：第24000張照片
目前進度：第24500張照片
目前進度：第25000張照片
目前進度

### 篩除第二階段輸入的訓練資料中全黑的照片(對腎臟腫瘤的預測毫無幫助)

在跑這一段的程式碼之前，請先準備好腎臟腫瘤標記的資料，因為要同時對全黑的照片和其標記一同篩除。

In [25]:
train_2nd_dir_img = 'DATA_0930\\WW300WL50\\train\\images_only_containing_predicted_roi_tumor as kidney'
train_2nd_dir_anno = 'DATA_0930\\WW300WL50\\train\\annotations_tumor only(tumor as 1)'

In [36]:
from tqdm import tqdm

inps = glob.glob(os.path.join(train_2nd_dir_img, "*.png"))

for inp in tqdm(inps):
    img = cv2.imread(inp, cv2.IMREAD_GRAYSCALE)
    if np.sum(img) == 0:
        os.remove(inp)
        os.remove(os.path.join(train_2nd_dir_anno, inp.split('\\')[-1]))

花費時間(秒)：135.30567741394043
